In [14]:
import os
import sys
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score

In [15]:
from xgboost import XGBClassifier

from utils import generate_description
from pickle_job import sauvegarder_modele

In [16]:
df_pure = pd.read_csv(folder_path + '/SBA_Cleaned_Pure.csv')
df_pure = df_pure.drop(['Name', 'ApprovalDate', 'ApprovalFY'], axis=1)  # Drop multiple columns
df_pure.dropna(inplace=True)

In [17]:
generate_description(df_pure)

,feature,data_type,null,nulPct,unique,uniqueSample
0,City,object,0,0.0,28860,"[MIAMI, Kalona, N. PROVIDENCE,, HUDSON, NEW YORK]"
1,State,object,0,0.0,51,"[AZ, NY, NJ, TX]"
2,Bank,object,0,0.0,5221,"[PACIFIC WESTERN BANK, TD BANK, NATIONAL ASSOC..."
3,BankState,object,0,0.0,55,"[NC, TX, PA]"
4,NAICS,object,0,0.0,20,"[42, 72, 31-33]"
5,Term,int64,0,0.0,378,"[84, 350, 65, 33]"
6,NoEmp,int64,0,0.0,502,"[5, 1, 4, 28, 3]"
7,NewExist,bool,0,0.0,2,[True]
8,CreateJob,int64,0,0.0,218,"[0, 1]"
9,RetainedJob,int64,0,0.0,325,"[7, 11, 0, 6]"


In [18]:
y = df_pure['Approve']
X = df_pure.drop('Approve', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.05, random_state=42, stratify=y)

In [19]:
numerical_columns_selector = X.select_dtypes(exclude=['object', 'bool']).columns
categorical_columns_selector = X.select_dtypes(include=['object', 'bool']).columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

In [20]:
categorical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [21]:
numerical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [22]:
preprocessor = ColumnTransformer(
    transformers=[
        ("nums", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ],
    remainder='passthrough'
)

In [23]:
model = XGBClassifier()

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', model)])

In [24]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('nums',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Term', 'NoEmp', 'CreateJob', 'RetainedJob', 'GrAppv', 'SBA_Appv',
       'Franchise'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missi...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [25]:
y_pred = pipeline.predict(X_test)

In [26]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy : {accuracy}')

f1_macro = f1_score(y_test, y_pred, average="macro")
print(f"Score F1 : {f1_macro}")

print(classification_report(y_test, y_pred))

Accuracy : 0.9492377042120073
Score F1 : 0.9217572834931513
              precision    recall  f1-score   support

       False       0.91      0.85      0.88      7344
        True       0.96      0.98      0.97     27485

    accuracy                           0.95     34829
   macro avg       0.93      0.91      0.92     34829
weighted avg       0.95      0.95      0.95     34829



In [27]:
sauvegarder_modele(model, 'xgboost_model')

Le modèle a été sauvegardé dans '../Model/xgboost_model.pkl'.
